In [200]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
import pandas as pd
import numpy as np
import random

In [201]:
from datetime import datetime
df = pd.read_csv('D:\STUDIES\BTP\data.csv')
df['GENERATION_TIME'] = [ datetime.strptime(i[2:],'%y.%m.%d %H:%M') for i in df['GENERATION_TIME']]
df = df[['GENERATION_TIME','LATITUDE','LONGITUDE','OZONE_PPB']]

In [202]:
df['GENERATION_TIME'] = df['GENERATION_TIME'].dt.round('15min')
df['LATITUDE'] = round(df['LATITUDE'],3)
df['LONGITUDE'] = round(df['LONGITUDE'],3)
df = df.sort_values(by=['GENERATION_TIME','LATITUDE','LONGITUDE'])

In [203]:
pdf = df[(df['GENERATION_TIME'].dt.day<2)]
pdff = pdf[['GENERATION_TIME','LATITUDE','LONGITUDE']]

In [204]:
vals =[]
sums=pdf['OZONE_PPB'][0]
count=1
for i in range(1,len(pdf)):
    if(sum(pdff.iloc[i]==pdff.iloc[i-1]) == 3):
        sums += pdf['OZONE_PPB'][i]
        count += 1
    else:
        vals.append(round(sums/count,2))
        sums = pdf['OZONE_PPB'][i]
        count = 1
vals.append(sums/count)

In [205]:
df2 = pdff.drop_duplicates()
df2 = df2.assign(OZONE_PPB = vals)
#df2

In [206]:
num = [ len(df2[(df2['GENERATION_TIME'].dt.hour == i)]) for i in range(24) ]
print(np.where(np.array(num) == np.max(np.array(num)))[0][0])
print(num)

8
[4, 4, 4, 4, 10, 59, 55, 62, 72, 58, 58, 57, 49, 69, 59, 61, 62, 58, 55, 48, 11, 4, 4, 4]


In [310]:
data = df2[(df2['GENERATION_TIME'].dt.hour == 8)]
data = data[['LATITUDE','LONGITUDE','OZONE_PPB']]
data = data.to_numpy()
data = np.delete(data,[0,1,2],0)
testSet = random.sample(range(data.shape[0]),3)
testData = data[testSet]
data = np.delete(data,testSet,0)

In [311]:
edges = []
dist = 0.0035
for i in range(len(data)):
    count = 0
    for j in range(len(data)):
        if(i!=j):
            if((abs(data[i][0]-data[j][0])<dist) & (abs(data[i][1]-data[j][1])<dist)):
                edges.append([i,j])
                count+=1
        else:
            edges.append([i,i])
    if(count == 0):
        print(i)
    if(count == 1):
        print(i)
edges = np.array(edges).T

In [312]:
from torch_geometric.data import Data
def createDataset(d, edges):
    #d is OZONE_PPB values of train set, edges are edge list on this train set
    #d = torch.tensor(d, dtype = torch.float)
    #edges = torch.tensor(edges, dtype = torch.long)
    size = len(d)
    num = range(size)
    dataset = []
    trainSamples = int(size/20)
    count = 0
    while(count<50):
        trainSet = random.sample(num, trainSamples)
        subEdges = edges[:,[i not in trainSet for i in edges[0]]]
        subEdges = torch.tensor(subEdges, dtype =torch.long)
        subNodes = d*1.0
        subNodes[trainSet] = 0
        subNodes = np.reshape(subNodes, (subNodes.shape[0],1))
        subNodes = torch.tensor(subNodes, dtype = torch.float)
        sample = Data(x = subNodes, edge_index = subEdges, train_set = trainSet)
        if((not sample.contains_isolated_nodes()) and sample.is_coalesced()):
            dataset.append(sample)
            count+=1
    return dataset

In [313]:
X = data[:,2]
X1 = torch.reshape(torch.tensor(X),(-1,))
dataset = createDataset(X,edges)

In [314]:
X1

tensor([7.5700, 7.5200, 7.4600, 7.5700, 7.5700, 7.4900, 7.5900, 7.6000, 7.7200,
        7.6600, 7.2300, 7.3300, 7.4000, 7.4000, 7.4000, 7.4500, 7.5000, 7.6000,
        7.7200, 7.7600, 7.8700, 7.8600, 7.8600, 7.8600, 7.8900, 7.8200, 7.7400,
        7.8200, 7.9300, 7.9400, 7.9900, 8.0000, 8.0000, 8.0800, 8.1100, 8.0300,
        8.0000, 8.0800, 8.0900, 8.1500, 8.3400, 8.4200, 8.2500, 8.1500, 8.3100,
        8.4200, 8.7200, 9.1100, 9.2300, 9.3000, 9.4400, 9.5200, 9.5900, 9.6600,
        9.6800, 9.6600, 9.6600, 9.5900, 9.5200, 9.6600, 9.8400, 9.5600, 9.4900,
        9.6600, 9.5900, 9.6200], dtype=torch.float64)

r"""The GraphSAGE operator from the `"Inductive Representation Learning on
Large Graphs" <https://arxiv.org/abs/1706.02216>`_ paper
$$\mathbf{x}^{\prime}_i = \mathbf{W}_1 \mathbf{x}_i + \mathbf{W_2} \cdot
\mathrm{mean}_{j \in \mathcal{N(i)}} \mathbf{x}_j$$
Args:
    in_channels (int or tuple): Size of each input sample. A tuple
        corresponds to the sizes of source and target dimensionalities.
    out_channels (int): Size of each output sample.
    normalize (bool, optional): If set to :obj:`True`, output features
        will be :math:$\ell_2$-normalized, *i.e.*,
        :math:
        $$\frac{\mathbf{x}^{\prime}_i}
        {\| \mathbf{x}^{\prime}_i \|_2}$$.
        (default: :obj:`False`)
    bias (bool, optional): If set to :obj:`False`, the layer will not learn
        an additive bias. (default: :obj:`True`)
    **kwargs (optional): Additional arguments of
        :class:`torch_geometric.nn.conv.MessagePassing`.
"""

In [321]:
from torch_geometric.nn import SAGEConv
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = SAGEConv(1,3)       
        self.conv2 = SAGEConv(3,4)
        #self.conv3 = nn.Linear(3, 3)
        #self.batch3 = nn.BatchNorm1d(3)
        self.conv4 = nn.Linear(4, 1)

    def forward(self,data):
        x, edge_index = data.x.float(), data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        #x = F.relu(self.batch3(self.conv3(x)))
        x = self.conv4(x)
        return torch.squeeze(x)

net = Net()
net = net.float()
net.train()

Net(
  (conv1): SAGEConv(1, 3)
  (conv2): SAGEConv(3, 4)
  (conv4): Linear(in_features=4, out_features=1, bias=True)
)

In [322]:
net.conv1

SAGEConv(1, 3)

In [323]:
def train(opt):
    net.train()
    opt.zero_grad()
    fv = dataset[0]
    output = net(fv)
    output = torch.reshape(output,(-1,))
    loss = F.mse_loss(output[fv.train_set].float(),X1[fv.train_set].float())
    for i in dataset[1:]:
        output = net(i)
        output = torch.reshape(output,(-1,))
        indLoss = F.mse_loss(output.float(),X1.float())
        loss = loss + indLoss
    loss.backward()
    opt.step()
    return loss.item()

In [324]:
import torch.optim as optim
optimizer1 = torch.optim.Adam(net.parameters(), lr = 0.1)
optimizer01 = torch.optim.Adam(net.parameters(), lr = 0.01)
optimizer001 = torch.optim.Adam(net.parameters(), lr = 0.001)

In [325]:
for epoch in range(10000):
    loss = train(optimizer01)
    print(loss,epoch)
    #print(net(dataset[0]))
    if(loss<50):
        break

3494.672607421875 0
3305.259033203125 1
3129.776611328125 2
2958.10498046875 3
2786.779541015625 4
2615.400146484375 5
2443.970947265625 6
2272.541259765625 7
2101.148681640625 8
1929.86328125 9
1758.882080078125 10
1588.6290283203125 11
1419.799072265625 12
1253.35302734375 13
1090.4896240234375 14
932.6198120117188 15
781.3391723632812 16
638.4042358398438 17
505.699951171875 18
385.1917724609375 19
278.85400390625 20
188.56297302246094 21
115.94552612304688 22
62.17605209350586 23
27.722074508666992 24


In [326]:
for epoch in range(10000):
    loss = train(optimizer001)
    print(loss,epoch)
    print(net(dataset[0]))
    if(loss<6):
        break

12.060117721557617 0
tensor([7.5851, 7.5861, 7.5783, 6.6243, 7.6129, 6.6337, 7.6581, 7.6594, 7.6791,
        7.5700, 7.4642, 7.4723, 7.4664, 7.4799, 7.4799, 7.4907, 7.6863, 7.6992,
        7.7825, 7.7777, 7.8434, 7.8477, 7.8564, 7.8487, 7.8643, 7.8614, 7.8501,
        7.8738, 7.9036, 7.9760, 8.0001, 8.0359, 8.0359, 8.1676, 8.2314, 8.3884,
        8.4694, 8.5274, 8.5010, 8.4916, 8.4927, 8.5474, 8.5021, 8.5170, 8.6616,
        8.9672, 8.8657, 9.1440, 9.2357, 9.1751, 9.2967, 9.3064, 9.3194, 9.3349,
        8.7067, 8.5992, 7.4346, 8.7227, 8.6453, 8.8362, 9.0106, 9.2022, 9.1996,
        9.2005, 9.3100, 9.3298], grad_fn=<SqueezeBackward0>)
11.320024490356445 1
tensor([7.6333, 7.6343, 7.6264, 6.6647, 7.6612, 6.6741, 7.7068, 7.7081, 7.7279,
        7.6181, 7.5116, 7.5197, 7.5138, 7.5274, 7.5274, 7.5383, 7.7351, 7.7481,
        7.8320, 7.8272, 7.8932, 7.8976, 7.9064, 7.8986, 7.9143, 7.9114, 7.9000,
        7.9239, 7.9538, 8.0267, 8.0510, 8.0870, 8.0870, 8.2196, 8.2838, 8.4417,
        8.5232, 8

        9.3106, 9.4214, 9.4414], grad_fn=<SqueezeBackward0>)
10.676902770996094 13
tensor([7.6610, 7.6620, 7.6542, 6.6948, 7.6890, 6.7043, 7.7347, 7.7360, 7.7558,
        7.6456, 7.5390, 7.5471, 7.5411, 7.5547, 7.5547, 7.5656, 7.7633, 7.7763,
        7.8604, 7.8555, 7.9218, 7.9262, 7.9350, 7.9272, 7.9429, 7.9401, 7.9287,
        7.9526, 7.9826, 8.0559, 8.0803, 8.1164, 8.1164, 8.2494, 8.3140, 8.4727,
        8.5545, 8.6131, 8.5866, 8.5770, 8.5780, 8.6332, 8.5876, 8.6027, 8.7488,
        9.0575, 8.9549, 9.2358, 9.3285, 9.2672, 9.3900, 9.3997, 9.4128, 9.4285,
        8.7934, 8.6847, 7.5139, 8.8095, 8.7315, 8.9244, 9.1005, 9.2944, 9.2919,
        9.2927, 9.4033, 9.4233], grad_fn=<SqueezeBackward0>)
10.704009056091309 14
tensor([7.6503, 7.6513, 7.6435, 6.6867, 7.6783, 6.6962, 7.7240, 7.7253, 7.7449,
        7.6349, 7.5285, 7.5366, 7.5306, 7.5442, 7.5442, 7.5551, 7.7525, 7.7655,
        7.8495, 7.8446, 7.9107, 7.9151, 7.9239, 7.9162, 7.9318, 7.9290, 7.9177,
        7.9416, 7.9715, 8.0446, 8.

        9.3542, 9.4656, 9.4858], grad_fn=<SqueezeBackward0>)
10.638566970825195 26
tensor([7.7126, 7.7137, 7.7059, 6.7501, 7.7409, 6.7597, 7.7869, 7.7882, 7.8079,
        7.6970, 7.5902, 7.5982, 7.5920, 7.6057, 7.6057, 7.6166, 7.8160, 7.8289,
        7.9135, 7.9086, 7.9751, 7.9796, 7.9885, 7.9806, 7.9965, 7.9937, 7.9824,
        8.0064, 8.0365, 8.1103, 8.1348, 8.1714, 8.1714, 8.3052, 8.3704, 8.5305,
        8.6130, 8.6719, 8.6452, 8.6354, 8.6363, 8.6918, 8.6461, 8.6614, 8.8087,
        9.1196, 9.0158, 9.2983, 9.3915, 9.3298, 9.4532, 9.4630, 9.4761, 9.4918,
        8.8513, 8.7418, 7.5760, 8.8676, 8.7892, 8.9836, 9.1609, 9.3566, 9.3544,
        9.3550, 9.4664, 9.4866], grad_fn=<SqueezeBackward0>)
10.634778022766113 27
tensor([7.7106, 7.7116, 7.7039, 6.7494, 7.7389, 6.7591, 7.7849, 7.7862, 7.8058,
        7.6949, 7.5882, 7.5962, 7.5900, 7.6037, 7.6037, 7.6146, 7.8139, 7.8269,
        7.9114, 7.9065, 7.9730, 7.9774, 7.9863, 7.9785, 7.9943, 7.9915, 7.9803,
        8.0042, 8.0343, 8.1081, 8.

        9.3089, 9.4198, 9.4400], grad_fn=<SqueezeBackward0>)
10.521242141723633 39
tensor([7.6800, 7.6812, 7.6736, 6.7373, 7.7083, 6.7469, 7.7541, 7.7554, 7.7746,
        7.6642, 7.5587, 7.5664, 7.5601, 7.5737, 7.5737, 7.5845, 7.7834, 7.7961,
        7.8802, 7.8751, 7.9412, 7.9457, 7.9546, 7.9467, 7.9625, 7.9599, 7.9488,
        7.9726, 8.0024, 8.0761, 8.1004, 8.1369, 8.1369, 8.2701, 8.3353, 8.4951,
        8.5773, 8.6358, 8.6095, 8.5995, 8.6001, 8.6551, 8.6101, 8.6254, 8.7723,
        9.0819, 8.9780, 9.2586, 9.3514, 9.2899, 9.4125, 9.4220, 9.4350, 9.4505,
        8.8114, 8.7021, 7.5613, 8.8276, 8.7499, 8.9438, 9.1202, 9.3157, 9.3140,
        9.3142, 9.4252, 9.4454], grad_fn=<SqueezeBackward0>)
10.510174751281738 40
tensor([7.6847, 7.6859, 7.6782, 6.7424, 7.7130, 6.7520, 7.7588, 7.7601, 7.7793,
        7.6688, 7.5633, 7.5710, 7.5647, 7.5783, 7.5783, 7.5891, 7.7882, 7.8009,
        7.8850, 7.8799, 7.9460, 7.9505, 7.9594, 7.9515, 7.9673, 7.9647, 7.9537,
        7.9774, 8.0072, 8.0810, 8.

        9.3132, 9.4242, 9.4445], grad_fn=<SqueezeBackward0>)
10.422340393066406 52
tensor([7.6786, 7.6799, 7.6724, 6.7526, 7.7070, 6.7623, 7.7527, 7.7540, 7.7729,
        7.6625, 7.5579, 7.5653, 7.5589, 7.5724, 7.5724, 7.5831, 7.7825, 7.7950,
        7.8788, 7.8737, 7.9396, 7.9441, 7.9530, 7.9451, 7.9609, 7.9585, 7.9476,
        7.9712, 8.0008, 8.0747, 8.0989, 8.1356, 8.1356, 8.2686, 8.3341, 8.4944,
        8.5767, 8.6350, 8.6088, 8.5986, 8.5989, 8.6538, 8.6092, 8.6247, 8.7718,
        9.0814, 8.9768, 9.2566, 9.3494, 9.2879, 9.4102, 9.4195, 9.4324, 9.4477,
        8.8072, 8.6978, 7.5783, 8.8234, 8.7462, 8.9402, 9.1165, 9.3128, 9.3117,
        9.3114, 9.4224, 9.4427], grad_fn=<SqueezeBackward0>)
10.415657997131348 53
tensor([7.6782, 7.6795, 7.6720, 6.7536, 7.7066, 6.7633, 7.7523, 7.7536, 7.7724,
        7.6620, 7.5575, 7.5649, 7.5585, 7.5720, 7.5720, 7.5827, 7.7822, 7.7946,
        7.8784, 7.8733, 7.9392, 7.9436, 7.9525, 7.9447, 7.9605, 7.9580, 7.9472,
        7.9708, 8.0004, 8.0742, 8.

...

        9.3793, 9.4982, 9.5307], grad_fn=<SqueezeBackward0>)
7.648091793060303 3822
tensor([7.5179, 7.5250, 7.5239, 7.4039, 7.5583, 7.4203, 7.6075, 7.6077, 7.6057,
        7.4812, 7.4125, 7.4051, 7.3878, 7.3990, 7.3990, 7.4067, 7.6751, 7.6769,
        7.7599, 7.7483, 7.8111, 7.8172, 7.8306, 7.8202, 7.8420, 7.8479, 7.8478,
        7.8678, 7.8930, 7.9901, 8.0144, 8.0700, 8.0700, 8.2093, 8.3117, 8.5262,
        8.6201, 8.6775, 8.6654, 8.6397, 8.6283, 8.6765, 8.6567, 8.6769, 8.8742,
        9.2281, 9.0780, 9.3508, 9.4584, 9.3978, 9.5151, 9.5134, 9.5229, 9.5315,
        8.6944, 8.5559, 8.4742, 8.7051, 8.6498, 8.8988, 9.0984, 9.3590, 9.4012,
        9.3793, 9.4983, 9.5307], grad_fn=<SqueezeBackward0>)
7.648089408874512 3823
tensor([7.5178, 7.5250, 7.5239, 7.4039, 7.5583, 7.4202, 7.6075, 7.6076, 7.6057,
        7.4812, 7.4124, 7.4051, 7.3877, 7.3990, 7.3990, 7.4067, 7.6751, 7.6769,
        7.7599, 7.7483, 7.8111, 7.8172, 7.8306, 7.8201, 7.8419, 7.8479, 7.8478,
        7.8678, 7.8930, 7.9901, 

        9.3793, 9.4982, 9.5307], grad_fn=<SqueezeBackward0>)
7.648088455200195 3835
tensor([7.5179, 7.5250, 7.5239, 7.4039, 7.5583, 7.4202, 7.6075, 7.6076, 7.6057,
        7.4812, 7.4125, 7.4051, 7.3878, 7.3990, 7.3990, 7.4067, 7.6751, 7.6769,
        7.7599, 7.7483, 7.8111, 7.8172, 7.8306, 7.8202, 7.8420, 7.8479, 7.8478,
        7.8678, 7.8930, 7.9901, 8.0144, 8.0700, 8.0700, 8.2093, 8.3117, 8.5262,
        8.6201, 8.6775, 8.6653, 8.6397, 8.6283, 8.6764, 8.6567, 8.6769, 8.8742,
        9.2281, 9.0780, 9.3508, 9.4584, 9.3978, 9.5151, 9.5134, 9.5229, 9.5315,
        8.6943, 8.5559, 8.4742, 8.7051, 8.6498, 8.8988, 9.0984, 9.3590, 9.4012,
        9.3793, 9.4982, 9.5307], grad_fn=<SqueezeBackward0>)
7.6480865478515625 3836
tensor([7.5178, 7.5250, 7.5239, 7.4039, 7.5583, 7.4202, 7.6075, 7.6076, 7.6057,
        7.4812, 7.4125, 7.4051, 7.3878, 7.3990, 7.3990, 7.4067, 7.6751, 7.6769,
        7.7599, 7.7483, 7.8111, 7.8172, 7.8306, 7.8202, 7.8419, 7.8479, 7.8478,
        7.8678, 7.8930, 7.9901,

        9.3793, 9.4982, 9.5307], grad_fn=<SqueezeBackward0>)
7.648083209991455 3848
tensor([7.5178, 7.5249, 7.5239, 7.4039, 7.5583, 7.4202, 7.6074, 7.6076, 7.6057,
        7.4812, 7.4124, 7.4051, 7.3877, 7.3990, 7.3990, 7.4067, 7.6751, 7.6769,
        7.7599, 7.7483, 7.8111, 7.8172, 7.8306, 7.8201, 7.8419, 7.8479, 7.8478,
        7.8678, 7.8930, 7.9901, 8.0144, 8.0700, 8.0700, 8.2093, 8.3116, 8.5262,
        8.6201, 8.6775, 8.6653, 8.6397, 8.6282, 8.6764, 8.6567, 8.6769, 8.8742,
        9.2281, 9.0779, 9.3507, 9.4583, 9.3978, 9.5151, 9.5134, 9.5229, 9.5314,
        8.6943, 8.5559, 8.4741, 8.7050, 8.6498, 8.8988, 9.0984, 9.3590, 9.4012,
        9.3793, 9.4982, 9.5307], grad_fn=<SqueezeBackward0>)
7.648083209991455 3849
tensor([7.5179, 7.5250, 7.5239, 7.4039, 7.5583, 7.4203, 7.6075, 7.6077, 7.6057,
        7.4812, 7.4125, 7.4051, 7.3878, 7.3990, 7.3990, 7.4067, 7.6751, 7.6769,
        7.7599, 7.7483, 7.8111, 7.8173, 7.8306, 7.8202, 7.8420, 7.8480, 7.8478,
        7.8679, 7.8930, 7.9901, 

        9.3793, 9.4982, 9.5307], grad_fn=<SqueezeBackward0>)
7.648078441619873 3861
tensor([7.5179, 7.5250, 7.5239, 7.4040, 7.5584, 7.4203, 7.6075, 7.6077, 7.6057,
        7.4812, 7.4125, 7.4051, 7.3878, 7.3991, 7.3991, 7.4067, 7.6751, 7.6770,
        7.7599, 7.7483, 7.8111, 7.8173, 7.8306, 7.8202, 7.8420, 7.8480, 7.8478,
        7.8679, 7.8930, 7.9902, 8.0145, 8.0700, 8.0700, 8.2093, 8.3117, 8.5262,
        8.6201, 8.6775, 8.6654, 8.6397, 8.6283, 8.6765, 8.6567, 8.6769, 8.8743,
        9.2282, 9.0780, 9.3508, 9.4584, 9.3978, 9.5151, 9.5134, 9.5229, 9.5315,
        8.6944, 8.5560, 8.4742, 8.7051, 8.6498, 8.8989, 9.0984, 9.3590, 9.4013,
        9.3793, 9.4983, 9.5308], grad_fn=<SqueezeBackward0>)
7.648078441619873 3862
tensor([7.5178, 7.5249, 7.5238, 7.4039, 7.5583, 7.4202, 7.6074, 7.6076, 7.6057,
        7.4812, 7.4124, 7.4050, 7.3877, 7.3990, 7.3990, 7.4067, 7.6751, 7.6769,
        7.7599, 7.7482, 7.8111, 7.8172, 7.8305, 7.8201, 7.8419, 7.8479, 7.8477,
        7.8678, 7.8929, 7.9901, 

KeyboardInterrupt: 

In [327]:
def Meaner(X,edges):
    predVal = np.zeros((X.shape[0],2), dtype = float)
    for i in range(edges.shape[1]):
        predVal[edges[1][i]][0]+=X[edges[0][i]]
        predVal[edges[1][i]][1]+=1
    return predVal[:,0]/predVal[:,1]

outMean = torch.from_numpy(Meaner(X1,edges))
sumNN, sumMean = 0, 0
for i in range(50):
    out = net(dataset[i])
    out = torch.reshape(out,(-1,))
    lossOut = F.mse_loss(out, X1)
    lossOutVal = F.mse_loss(out[dataset[i].train_set], X1[dataset[i].train_set])
    meanLossOut = F.mse_loss(outMean, X1)
    meanLossOutVal = F.mse_loss(outMean[dataset[i].train_set], X1[dataset[i].train_set])
    sumNN+=lossOutVal
    sumMean+=meanLossOutVal
#     print("Iter ", i)
#     print("On Whole Set: ", lossOut.item(),"(NN), ", meanLossOut.item(),"(Just Mean)")
#     print("On Train Set: ", lossOutVal.item(),"(NN), ", meanLossOutVal.item(),"(Just Mean)")
#     print(" ")
print("Total Loss by NN: ", sumNN)
print("Total Loss by MeanEstimate: ", sumMean)

Total Loss by NN:  tensor(11.5203, dtype=torch.float64, grad_fn=<AddBackward0>)
Total Loss by MeanEstimate:  tensor(7.3095, dtype=torch.float64)


In [328]:
def prepareTestData(trainData, testData):
    finalData = np.concatenate((trainData,testData), axis = 0)
    finalEdges = edges.T
    trainLen = len(trainData)
    newEdges = np.zeros((0,2), dtype = int)
    for i in range(len(testData)):
        for j in range(len(trainData)):
            if(i!=j):
                if((abs(trainData[j][0]-testData[i][0])<dist) & (abs(trainData[j][1]-testData[i][1])<dist)):
                    newEdges = np.concatenate((newEdges, np.array([[j,trainLen+i]])), axis = 0)
            else:
                1#newEdges = np.concatenate((newEdges, np.array([[trainLen+i,trainLen+i]])), axis = 0)
    finalEdges = np.concatenate((finalEdges, newEdges), axis = 0)
    return finalData[:,2],finalEdges.T

In [329]:
Xf, ef = prepareTestData(data, testData)
Xf[-(len(testData)):] = 0
Xf = np.reshape(Xf,(Xf.shape[0],1))
test_set = range(len(X),len(Xf))
model = Data(x = torch.tensor(Xf, dtype= torch.float), edge_index = torch.tensor(ef, dtype= torch.long), test_set = test_set)
net.eval()
print("Loss by GNN model: ", F.mse_loss(net(model)[test_set], torch.tensor(testData[:,2])))
print("Loss by mean model: ", F.mse_loss(torch.tensor(Meaner(Xf,ef)[test_set]), torch.tensor(testData[:,2])))

Loss by GNN model:  tensor(0.1755, dtype=torch.float64, grad_fn=<MseLossBackward>)
Loss by mean model:  tensor(0.2267, dtype=torch.float64)


In [330]:
net(model)[test_set], testData[:,2], Meaner(Xf,ef)[test_set]

(tensor([8.8514, 7.3706, 8.9189], grad_fn=<IndexBackward>),
 array([8.41, 7.46, 8.35]),
 array([8.92571429, 7.475     , 8.99333333]))

In [331]:
net(model),X,testData

(tensor([7.5265, 7.5281, 7.5270, 7.5371, 7.5448, 7.5433, 7.5754, 7.5820, 7.5959,
         7.4812, 7.4124, 7.4050, 7.3877, 7.3990, 7.3990, 7.4067, 7.6751, 7.6769,
         7.7598, 7.7482, 7.8110, 7.8172, 7.8305, 7.8201, 7.8419, 7.8479, 7.8477,
         7.8678, 7.8929, 7.9901, 8.0144, 8.0902, 8.0902, 8.2497, 8.4549, 8.6627,
         8.7466, 8.7976, 8.7425, 8.6598, 8.6459, 8.6905, 8.6566, 8.6768, 8.8742,
         9.2281, 9.0779, 9.3507, 9.4583, 9.3977, 9.5150, 9.5133, 9.5228, 9.5314,
         8.7715, 8.6924, 8.7769, 8.8251, 8.6674, 8.8988, 9.0984, 9.3589, 9.4012,
         9.3792, 9.4982, 9.5307, 8.8514, 7.3706, 8.9189],
        grad_fn=<SqueezeBackward0>),
 array([7.57, 7.52, 7.46, 7.57, 7.57, 7.49, 7.59, 7.6 , 7.72, 7.66, 7.23,
        7.33, 7.4 , 7.4 , 7.4 , 7.45, 7.5 , 7.6 , 7.72, 7.76, 7.87, 7.86,
        7.86, 7.86, 7.89, 7.82, 7.74, 7.82, 7.93, 7.94, 7.99, 8.  , 8.  ,
        8.08, 8.11, 8.03, 8.  , 8.08, 8.09, 8.15, 8.34, 8.42, 8.25, 8.15,
        8.31, 8.42, 8.72, 9.11, 9.23, 9.3 

In [332]:
testSet

[46, 16, 48]